# Step 5: Machine Learning

In [ ]:
from ml_tools.ensemble_learning import RegressionTreeModels, run_ensemble_pipeline
from paths import MICE_DATASETS_DIR, VIF_DATASETS_DIR, FEATURE_ENG_DATASETS_DIR, ENSEMBLE_RESULTS_DIR, TRAIN_DATASETS_DIR, LOGS_DIR
from helpers.constants import TARGETS
from ml_tools.logger import custom_logger
from ml_tools.utilities import train_dataset_orchestrator

## 1. Create Training Datasets

In [ ]:
train_dataset_orchestrator(list_of_dirs=[FEATURE_ENG_DATASETS_DIR, MICE_DATASETS_DIR, VIF_DATASETS_DIR],
                           target_columns=TARGETS,
                           save_dir=TRAIN_DATASETS_DIR)

## 2. Configure Model Hyperparameters

In [ ]:
hyperparameters = {
    "L1_regularization": 0,
    "L2_regularization": 0,
    "learning_rate": 0.001,
    "n_estimators": 3000,   #xgb - lightgbm
    "max_depth": 8,
    "subsample": 0.8,
    "colsample_bytree": 0.8,    #xgb - lightgbm
    "min_samples_leaf": 50, #histgb
    "max_iter": 1000,   #histgb
    "min_child_weight": 3,  #xgb
    "gamma": 1, #xgb
    "num_leaves": 31,   #lightgbm
    "min_data_in_leaf": 40  #lightgbm
}

factory_class = RegressionTreeModels(**hyperparameters)

## 3. Train Ensemble models

In [ ]:
run_ensemble_pipeline(datasets_dir=TRAIN_DATASETS_DIR,
                      save_dir=ENSEMBLE_RESULTS_DIR,
                      target_columns=TARGETS,
                      model_object=factory_class,
                      save_model=True)